---
# **LAB 5 - Unrolling e parallelismo dinamico**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

GPU computing notebooks download (from github)

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

NVCC Plugin for Jupyter notebook

In [ ]:
%cd GPUcomputing/utils/nvcc4jupyter-master/
!!python3 -m build
%load_ext nvcc4jupyter
%cd /content/

# ✅ Occupancy Calculator - DeviceQuery

In [ ]:
%%cuda
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  GPU arch name:                                 %s\n",
						_ConvertSMVer2ArchName(deviceProp.major, deviceProp.minor));

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

		printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
						deviceProp.multiProcessorCount,
						_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
						_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
								deviceProp.multiProcessorCount);

		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory                %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block        %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block  %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size                                      %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor   %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block            %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z)  (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z)  (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch                           %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment                              %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution           %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels                      %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory             %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping        %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces             %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support                         %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


Several API functions exist to assist programmers in choosing thread block size and cluster size based on register and shared memory requirements.

- `cudaOccupancyMaxActiveBlocksPerMultiprocessor`, is the occupancy calculator API that provides an **occupancy prediction based on the block size and shared memory usage** of a kernel. This function reports occupancy in terms of the number of concurrent thread blocks per multiprocessor.
Note that this value can be converted to other metrics. Multiplying by the number of warps per block yields the number of concurrent warps per multiprocessor; further dividing concurrent warps by max warps per multiprocessor gives the occupancy as a percentage.
- `cudaOccupancyMaxPotentialBlockSize` and `cudaOccupancyMaxPotentialBlockSizeVariableSMem`, are the occupancy-based launch configurator APIs, **heuristically calculate an execution configuration that achieves the maximum multiprocessor-level occupancy.**
- `cudaOccupancyMaxActiveClusters`, can provided occupancy prediction based on the cluster size, block size and shared memory usage of a kernel. This function reports occupancy in terms of number of max active clusters of a given size on the GPU present in the system.



---


The following code sample calculates the occupancy of MyKernel. It then reports the occupancy level with the **ratio between concurrent warps versus maximum warps per multiprocessor**.

In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "OCC"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"


// Device code
__global__ void MyKernel(int *d, int *a, int *b) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  d[idx] = a[idx] * b[idx];
}

// Host code
int main() {
  device_feat();    // device feats
  int numBlocks;    // number of of active blocks

  // These variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;

  for (int blockSize = 16; blockSize <= 1024; blockSize*=2) {
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, MyKernel, blockSize, 0);
    activeWarps = numBlocks * blockSize / prop.warpSize;
    double occup = (double)activeWarps / maxWarps * 100;
    printf("blockSize = %4d <-> Occupancy [numBlocks = %2d,  activeWarps = %2d]:\t%2.2f%%\n", blockSize, numBlocks, activeWarps, occup);
  }
  return 0;
}

In [ ]:
!nvcc -arch=sm_75 src/OCC/occupancy.cu -o occupancy
!./occupancy



---

The following code sample configures an occupancy-based kernel launch of MyKernel according to the user input.


In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "OCC"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"


// Device code
__global__ void MyKernel(float *array, int arrayCount) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if (idx < arrayCount)
    array[idx] *= array[idx];
}

__global__ void MyKernel1(int *d, int *a, int *b) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  d[idx] = a[idx] * b[idx];
}

// occupancy calculator
int occupancy(float *array, int arrayCount) {
  int blockSize;      // The launch configurator returned block size
  int minGridSize;    // The minimum grid size needed to achieve the
                      // maximum occupancy for a full device launch
  int gridSize;       // The actual grid size needed, based on input size

  cudaOccupancyMaxPotentialBlockSize(&minGridSize, &blockSize, (void*)MyKernel, 0, arrayCount);

  // Round up according to array size
  gridSize = (arrayCount + blockSize - 1) / blockSize;
  printf("blockSize = %4d, gridSize = %d, minGridSize = %2d\n", blockSize, gridSize, minGridSize);

  MyKernel<<<gridSize, blockSize>>>(array, arrayCount);
  cudaDeviceSynchronize();

  // compute occupancy
  int device;
  int numBlocks;
  cudaDeviceProp prop;
  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  int maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;
  cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, MyKernel, blockSize, 0);
  int activeWarps = numBlocks * blockSize / 32;
  double occup = (double)activeWarps / maxWarps * 100;
  printf("Occupancy [blockSize = %4d, activeWarps = %2d]:\t%2.2f%%\n", blockSize, activeWarps, occup);

  return 0;
}

// MAIN
int main(void) {
  device_feat();    // device feats
  int N = 2<<20;
  size_t size = N * sizeof(float);

  // Allocate input vectors h_A and h_B in host memory
  float* h_A = (float*)malloc(size);
  float* h_B = (float*)malloc(size);
  float* h_C = (float*)malloc(size);

  // Allocate vectors in device memory
  float* d_A;
  cudaMalloc(&d_A, size);
  float* d_B;
  cudaMalloc(&d_B, size);
  float* d_C;
  cudaMalloc(&d_C, size);

  // Copy vectors from host memory to device memory
  cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

  // Invoke kernel
  occupancy(d_A, N);

  // Free device memory
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  return 0;
}

In [ ]:
!nvcc -arch=sm_75  src/OCC/occupancy.cu -o occupancy
!./occupancy

### ↘️ *`TODO...`*

Apply the same calculator to the kernel:

```
__global__ void blockParReduce1(int *in, int *out, ulong n)
```



In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "OCC"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"

__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

// MAIN
int main(void) {
  int numBlocks;        // Occupancy in terms of active blocks

  // These variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;

  for (int blockSize = 16; blockSize <= 1024; blockSize*=2) {
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, blockParReduce1, blockSize, 0);
    activeWarps = numBlocks * blockSize / prop.warpSize;
    double occup = (double)activeWarps / maxWarps * 100;
    printf("Occupancy [blockSize = %4d, activeWarps = %2d]:\t%2.2f%%\n", blockSize, activeWarps, occup);
  }
  return 0;
}

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/OCC/occupancy.cu -o occupancy
!./occupancy

Apply the same calculator to the kernel:

```
__global__ void blockParReduce_SMEM(int *in, int *out, ulong n)
```


In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "OCC"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"

#define SMEM_DIM 1024

/*
    This version uses sequential addressing -- no divergence or bank conflicts.
*/
__global__ void blockParReduce_SMEM(int *in, int *out, ulong n) {

	// shared mem
	__shared__ int smem[SMEM_DIM];

	unsigned int tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// load shared mem
	smem[tid] = (idx < n) ? in[idx] : 0;

	// synchronize within threadblock
	__syncthreads();

	// do reduction in shared mem
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
		if (tid < stride)
			smem[tid] += smem[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = smem[0];
}


// MAIN
int main(void) {
  int numBlocks;        // Occupancy in terms of active blocks

  // These variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;

  for (int blockSize = 16; blockSize <= 1024; blockSize*=2) {
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, blockParReduce_SMEM, blockSize, 0);
    activeWarps = numBlocks * blockSize / prop.warpSize;
    double occup = (double)activeWarps / maxWarps * 100;
    printf("Occupancy [blockSize = %4d, activeWarps = %2d]:\t%2.2f%%\n", blockSize, activeWarps, occup);
  }
  return 0;
}

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/OCC/occupancy.cu -o occupancy
!./occupancy

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/OCC/occupancy.cu

# ✅ Profiling

In [ ]:
%%cuda_group_save --name "preduce.cu" --group "OCC"
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"


/*
 *  Block by block parallel implementation with divergence (sequential schema)
 */
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Block by block parallel implementation without divergence (interleaved schema)
 */
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)  {
		if (tid < stride)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}


/*
 * MAIN: test on parallel reduction
 */
int main(void) {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;            // block dim 1D
	ulong numBlock = 1024*1024;      // grid dim 1D
	ulong n = blockSize * numBlock;  // array dim
	long sum_CPU = 0, sum_GPU;
	long nByte = n*sizeof(int), mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	for (ulong i = 0; i < n; i++) a[i] = 1;

	CHECK(cudaMalloc((void **) &d_a, nByte));
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void **) &d_b, mByte));
	CHECK(cudaMemset((void *) d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++)
    sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);

  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i]=1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());

  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
  //		printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

  // reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	cudaFree(d_a);

	CHECK(cudaDeviceReset());
	return 0;
}

In [ ]:
!nvcc -O2 -arch=sm_75  src/OCC/preduce.cu -o preduce
!./preduce

!ncu preduce
!ncu -o preduce_profile preduce

# ✅ Parallel reduction unrolling




In [ ]:
%%cuda_group_save --name "preduceUNROLL.cu" --group "UNROLL"

#include <cuda_runtime.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/common.h"


#define DIM 1024

/*
 * An example of using shared memory to optimize performance of a parallel
 * reduction by constructing partial results for a thread block in shared memory
 * before flushing to global memory.
 */

extern __shared__ int dsmem[];

//# Recursive Implementation of Interleaved Pair Approach
int recursiveReduce(int *data, int const size) {
  if (size == 1) return data[0];

  int const stride = size / 2;

  for (int i = 0; i < stride; i++)
    data[i] += data[i + stride];

  return recursiveReduce(data, stride);
}

__device__ void warpReduce(volatile int *smem, unsigned int tid) {
  smem[tid] = min(smem[tid],smem[tid+32]);
  smem[tid] = min(smem[tid],smem[tid+16]);
  smem[tid] = min(smem[tid],smem[tid+8]);
  smem[tid] = min(smem[tid],smem[tid+4]);
  smem[tid] = min(smem[tid],smem[tid+2]);
  smem[tid] = min(smem[tid],smem[tid+1]);
}

//# unroll4 + complete unroll for loop + gmem
__global__ void reduceGmem(int *g_idata, int *g_odata, unsigned int n) {
    //# set thread ID
    unsigned int tid = threadIdx.x;
    int *idata = g_idata + blockIdx.x * blockDim.x;

    //# boundary check
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= n) return;

    //# in-place reduction in global memory
    if (blockDim.x >= 1024 && tid < 512) idata[tid] += idata[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256) idata[tid] += idata[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) idata[tid] += idata[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64) idata[tid] += idata[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32)
      warpReduce(idata, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = idata[0];
}

__global__ void reduceSmem(int *g_idata, int *g_odata, unsigned int n) {
    __shared__ int smem[DIM];

    // set thread ID
    unsigned int tid = threadIdx.x;

    // boundary check
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= n) return;

    // convert global data pointer to the local pointer of this block
    int *idata = g_idata + blockIdx.x * blockDim.x;

    // set to smem by each threads
    smem[tid] = idata[tid];
    __syncthreads();

    // in-place reduction in shared memory
    if (blockDim.x >= 1024 && tid < 512) smem[tid] += smem[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256) smem[tid] += smem[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64)  smem[tid] += smem[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32)
      warpReduce(smem, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}

__global__ void reduceSmemDyn(int *g_idata, int *g_odata, unsigned int n) {
    extern __shared__ int smem[];

    // set thread ID
    unsigned int tid = threadIdx.x;
    int *idata = g_idata + blockIdx.x * blockDim.x;

    // set to smem by each threads
    smem[tid] = idata[tid];
    __syncthreads();

    // in-place reduction in global memory
    if (blockDim.x >= 1024 && tid < 512)  smem[tid] += smem[tid + 512];

    __syncthreads();

    if (blockDim.x >= 512 && tid < 256)  smem[tid] += smem[tid + 256];

    __syncthreads();

    if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];

    __syncthreads();

    if (blockDim.x >= 128 && tid < 64) smem[tid] += smem[tid + 64];

    __syncthreads();

    // unrolling warp
    if (tid < 32)
      warpReduce(smem, tid);

    // write result for this block to global mem
    if (tid == 0) g_odata[blockIdx.x] = smem[0];
}


int main(int argc, char **argv) {
  // set up device
  int dev = 0;
  cudaDeviceProp deviceProp;
  CHECK(cudaGetDeviceProperties(&deviceProp, dev));
  printf("%s starting reduction at ", argv[0]);
  printf("device %d: %s ", dev, deviceProp.name);
  CHECK(cudaSetDevice(dev));

  bool bResult = false;

  // initialization
  int power = 10;

  // execution configuration
  int blocksize = DIM;   // initial block size

  if (argc >= 2)
      blocksize = atoi(argv[1]);

  if (argc >= 3)
      power = atoi(argv[2]);

  int size = 1 << power; // total number of elements to reduce
  printf("    with array size %d  ", size);

  dim3 block (blocksize, 1);
  dim3 grid  ((size + block.x - 1) / block.x, 1);
  printf("grid %d block %d\n", grid.x, block.x);

  // allocate host memory
  size_t bytes = size * sizeof(int);
  int *h_idata = (int *) malloc(bytes);
  int *h_odata = (int *) malloc(grid.x * sizeof(int));
  int *tmp     = (int *) malloc(bytes);

  // initialize the array
  for (int i = 0; i < size; i++)
    h_idata[i] = (int)( rand() & 0xFF );

  memcpy (tmp, h_idata, bytes);

  int gpu_sum = 0;

  // allocate device memory
  int *d_idata = NULL;
  int *d_odata = NULL;
  CHECK(cudaMalloc((void **) &d_idata, bytes));
  CHECK(cudaMalloc((void **) &d_odata, grid.x * sizeof(int)));

  // cpu reduction
  int cpu_sum = recursiveReduce (tmp, size);
  printf("cpu reduce          : %d\n", cpu_sum);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceNeighboredGmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++)
    gpu_sum += h_odata[i];

  printf("reduceNeighboredGmem: %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceNeighboredSmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceNeighboredSmem: %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceGmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceGmem          : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmem<<<grid.x, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceSmem          : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmemDyn<<<grid.x, block, blocksize*sizeof(int)>>>(d_idata, d_odata,
          size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x; i++) gpu_sum += h_odata[i];

  printf("reduceSmemDyn       : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x, block.x);

  // reduce gmem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceGmemUnroll<<<grid.x / 4, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x / 4 * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x / 4; i++) gpu_sum += h_odata[i];

  printf("reduceGmemUnroll4   : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x / 4, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmemUnroll<<<grid.x / 4, block>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x / 4 * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x / 4; i++) gpu_sum += h_odata[i];

  printf("reduceSmemUnroll4   : %d <<<grid %d block %d>>>\n", gpu_sum, grid.x / 4, block.x);

  // reduce smem
  CHECK(cudaMemcpy(d_idata, h_idata, bytes, cudaMemcpyHostToDevice));
  reduceSmemUnrollDyn<<<grid.x / 4, block, DIM*sizeof(int)>>>(d_idata, d_odata, size);
  CHECK(cudaGetLastError());
  CHECK(cudaMemcpy(h_odata, d_odata, grid.x / 4 * sizeof(int), cudaMemcpyDeviceToHost));
  gpu_sum = 0;

  for (int i = 0; i < grid.x / 4; i++) gpu_sum += h_odata[i];

  printf("reduceSmemDynUnroll4: %d <<<grid %d block %d>>>\n", gpu_sum, grid.x / 4, block.x);

  // free host memory
  free(h_idata);
  free(h_odata);

  // free device memory
  CHECK(cudaFree(d_idata));
  CHECK(cudaFree(d_odata));

  // reset device
  CHECK(cudaDeviceReset());

  // check the results
  bResult = (gpu_sum == cpu_sum);

  if(!bResult) printf("Test failed!\n");

  return EXIT_SUCCESS;
}


In [ ]:
!nvcc -arch=sm_75  src/UNROLL/preduceUNROLL.cu -o preduce
!./preduce 1024 20

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/preduceSMEM.cu -o preduce
!./preduce 1024 20

### ↘️ *`TODO...`*

Kernel privo di divergenza + unrolling:

* Introdurre warp unrolling
* Introdurre block unrolling
* Specializzare su diversi num blocchi
* Confrontare i vari kernel...




In [ ]:
%%cuda_group_save --name "preduceUnroll.cu" --group "OCC"

#include <assert.h>
#include "../GPUcomputing/utils/common.h"


/*
 *  Device function: block parallel reduction based on warp unrolling
 */
__device__ void blockWarpUnroll(int *thisBlock, int blockDim, uint tid) {
  // in-place reduction in global memory
    for (int stride = blockDim / 2; stride > 32; stride >>= 1)  {
      if (tid < stride)
        thisBlock[tid] += thisBlock[tid + stride];

      // synchronize within threadblock
      __syncthreads();
    }

    // unrolling warp
    if (tid < 32) {
      volatile int *vmem = thisBlock;
      vmem[tid] += vmem[tid + 32];
      vmem[tid] += vmem[tid + 16];
      vmem[tid] += vmem[tid + 8];
      vmem[tid] += vmem[tid + 4];
      vmem[tid] += vmem[tid + 2];
      vmem[tid] += vmem[tid + 1];
    }
}

/*
 *  Block by block parallel implementation with warp unrolling
 */
__global__ void blockParReduceUroll(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

  // block parall. reduction based on warp unrolling
  blockWarpUnroll(thisBlock, blockDim.x, tid);

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Multi block parallel implementation with block and warp unrolling
 */
__global__ void multBlockParReduceUroll8(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x * 8 + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x * 8;

    // unrolling 8 blocks
    if (idx + 7 * blockDim.x < n) {
        int a1 = in[idx];
        int a2 = in[idx + blockDim.x];
        int a3 = in[idx + 2 * blockDim.x];
        int a4 = in[idx + 3 * blockDim.x];
        int a5 = in[idx + 4 * blockDim.x];
        int a6 = in[idx + 5 * blockDim.x];
        int a7 = in[idx + 6 * blockDim.x];
        int a8 = in[idx + 7 * blockDim.x];
        in[idx] = a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8;
    }
    __syncthreads();

	// block parall. reduction based on warp unrolling
  blockWarpUnroll(thisBlock, blockDim.x, tid);

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Multi block parallel implementation with block and warp unrolling
 */
__global__ void multBlockParReduceUroll16(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x * 16 + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x * 16;

    // unrolling 16 blocks
    if (idx + 15 * blockDim.x < n) {
    	int a1 = in[idx];
    	int a2 = in[idx + blockDim.x];
    	int a3 = in[idx + 2 * blockDim.x];
    	int a4 = in[idx + 3 * blockDim.x];
    	int a5 = in[idx + 4 * blockDim.x];
    	int a6 = in[idx + 5 * blockDim.x];
    	int a7 = in[idx + 6 * blockDim.x];
    	int a8 = in[idx + 7 * blockDim.x];
    	int a9 = in[idx + 8 * blockDim.x];
    	int a10 = in[idx + 9 * blockDim.x];
    	int a11 = in[idx + 10 * blockDim.x];
    	int a12 = in[idx + 11 * blockDim.x];
    	int a13 = in[idx + 12 * blockDim.x];
    	int a14 = in[idx + 13 * blockDim.x];
    	int a15 = in[idx + 14 * blockDim.x];
    	int a16 = in[idx + 15 * blockDim.x];
    	in[idx] = a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8+
    			a9 + a10 + a11 + a12 + a13 + a14 + a15 + a16;
    }
    __syncthreads();

	// block parall. reduction based on warp unrolling
  blockWarpUnroll(thisBlock, blockDim.x, tid);

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Block by block parallel implementation without divergence (interleaved schema)
 */
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)  {
		if (tid < stride)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}



/*
 * MAIN: test on parallel reduction
 */
int main(void) {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;            // block dim 1D
	ulong numBlock = 1024*1024;      // grid dim 1D
	ulong n = blockSize * numBlock;  // array dim
	long sum_CPU = 0, sum_GPU;
	long nByte = n*sizeof(int), mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	CHECK(cudaMalloc((void **) &d_a, nByte));
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void **) &d_b, mByte));
	CHECK(cudaMemset((void *) d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f GB\n",n/(1024.0*1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++) sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
	}
	assert(sum_GPU == n);
  // reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*               KERNEL blockParReduceUroll                */
	/***********************************************************/

	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduceUroll...\n");
	start = seconds();
	blockParReduceUroll<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);
	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*            KERNEL multBlockParReduceUroll8              */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: multBlockParReduceUroll8...\n");
	start = seconds();

	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock/8; i++)
		sum_GPU += b[i];
	printf("    sum: %lu\n",sum_GPU);
	assert(sum_GPU == n);
	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*            KERNEL multBlockParReduceUroll16             */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: multBlockParReduceUroll16...\n");
	start = seconds();
	multBlockParReduceUroll16<<<numBlock/16, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock/16; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	cudaFree(d_a);

	CHECK(cudaDeviceReset());
	return 0;
}

In [ ]:
!nvcc -arch=sm_70 src/preduceUnroll.cu -o preduceUnroll
!./preduceUnroll

# ✅ Parallelismo dinamico

In [ ]:
%%cuda --name nestedHelloWorld.cu
#include <stdio.h>

/*
 * A simple example of nested kernel launches from the GPU. Each thread displays
 * its information when execution begins, and also diagnostics when the next
 * lowest nesting layer completes.
 */

__global__ void nestedHelloWorld(int const iSize, int iDepth) {
	int tid = threadIdx.x;
	printf("Recursion=%d: Hello World from thread %d block %d\n", iDepth, tid,	blockIdx.x);

	// condition to stop recursive execution
	if (iSize == 1)
		return;

	// reduce block size to half
	int nthreads = iSize >> 1;


	// thread 0 launches child grid recursively
	if (tid == 0 && nthreads > 0) {
		nestedHelloWorld<<<gridDim.x, nthreads>>>(nthreads, ++iDepth);
		printf("-------> nested execution depth: %d\n", iDepth);
	}
}

int main(int argc, char **argv) {
	int size = 8;
	int blocksize = 8;   // initial block size
	int igrid = 2;

	size = igrid * blocksize;

	dim3 block(blocksize, 1);
	dim3 grid((size + block.x - 1) / block.x, 1);
	printf("%s Execution Configuration: grid %d block %d\n", argv[0], grid.x,
			block.x);

	nestedHelloWorld<<<grid, block>>>(block.x, 0);

	cudaDeviceReset();
	return 0;
}

In [ ]:
!nvcc -arch=sm_70 -dc src/nestedHelloWorld.cu
!nvcc -arch=sm_70 *.o -o nestedHelloWorld
!./nestedHelloWorld


Introdurre il parallelismo dinamico nel calcolo di prodotti MQDB

In [ ]:
%%cuda --name mqdb.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <sys/time.h>
#include <time.h>

#ifndef MQDB_H
#define MQDB_H

#define randu() ((float)rand() / (float) RAND_MAX)
#define abs(x) ((x)<0 ? (-x) : (x))

typedef unsigned long ulong;
typedef unsigned int uint;

typedef struct MQDB {
	char desc[100];   // description
	int nBlocks;      // num. of blocks
	int *blkSize;     // block dimensions
	float *elem;       // elements in row-major order
	ulong nElems;     // actual number of elements
} mqdb;

typedef unsigned long ulong;
typedef unsigned int uint;

// # function prototypes #
int genRandDims(mqdb*, uint, uint);
void fillBlocks(mqdb*, uint, uint, char, float);
mqdb mqdbConst(uint, uint, uint, float);
void mqdbProd(mqdb, mqdb, mqdb);
void matProd(mqdb, mqdb, mqdb);
void checkResult(mqdb, mqdb);
void mqdbDisplay(mqdb);

#endif

In [ ]:
%%cuda --name MQDB-CUDA-DP.cu

#include "mqdb.h"
#include "../GPUcomputing/utils/common.h"

#define BLOCK_SIZE 16     // block size

struct tms {
	double GPUtmsMQDB;
	double GPUtmsMQDBDynPar1;
	double GPUtmsMQDBDynPark;
	float density;
};

// the kernels prototype
__global__ void mqdbBlockProd(mqdb, mqdb, mqdb, uint, uint, uint);
__global__ void mqdbProdDP1(mqdb, mqdb, mqdb, uint, uint);
__global__ void mqdbProdDPk(mqdb, mqdb, mqdb, uint);

/*
 * Test on MQDB kernels
 */
void testKernelsMQDB(uint n, uint k, struct tms* times) {

	// mqdb host matrices
	mqdb A, B, C, C1;

	// mqdb device matrices
	mqdb d_A, d_B, d_C;

	// fill in
	A = mqdbConst(n, k, 10, 1);
	B = mqdbConst(n, k, 10, 1);
	C = mqdbConst(n, k, 10, 1);
	C1 = mqdbConst(n, k, 10, 1);

	ulong nBytes = n * n * sizeof(float);
	ulong kBytes = k * sizeof(uint);
	printf("Memory size required = %.1f (MB)\n",(float)nBytes/(1024.0*1024.0));

	// malloc and copy on device memory
	d_A.nBlocks = A.nBlocks;
	CHECK(cudaMalloc((void**)&d_A.blkSize, kBytes));
	CHECK(cudaMemcpy(d_A.blkSize, A.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_A.elem, nBytes));
	CHECK(cudaMemcpy(d_A.elem, A.elem, nBytes, cudaMemcpyHostToDevice));
	d_B.nBlocks = B.nBlocks;
	CHECK(cudaMalloc((void**)&d_B.blkSize, kBytes));
	CHECK(cudaMemcpy(d_B.blkSize, B.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_B.elem, nBytes));
	CHECK(cudaMemcpy(d_B.elem, B.elem, nBytes, cudaMemcpyHostToDevice));
	d_C.nBlocks = C.nBlocks;
	CHECK(cudaMalloc((void**)&d_C.blkSize, kBytes));
	CHECK(cudaMemcpy(d_C.blkSize, C.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_C.elem, nBytes));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*                     GPU MQDB product                    */
	/***********************************************************/
	printf("Kernel MQDB product...\n");
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	uint sdim = 0;
	double start = seconds();
	for (uint i = 0; i < k; i++ ) {
		uint d = A.blkSize[i];
		mqdbBlockProd<<<grid, block>>>(d_A, d_B, d_C, sdim, d, n);
		sdim += d;
	}
	CHECK(cudaDeviceSynchronize());
	double GPUtime2 = seconds() - start;
	printf("   elapsed time:                    %.2f (sec)\n", GPUtime2);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*              GPU MQDB dynamic par. GRID(1)              */
	/***********************************************************/
	start = seconds();
	printf("Kernel MQDB product with dynamic parall. GRID(1)...\n");
	mqdbProdDP1<<< 1, 1 >>>(d_A, d_B, d_C, k, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime3 = seconds() - start;
	printf("   elapsed time:                        %.2f (sec)\n", GPUtime3);
	printf("   speedup vs GPU MQDB product:         %.2f\n", GPUtime2/GPUtime3);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*              GPU MQDB dynamic par. GRID(k)              */
	/***********************************************************/
	start = seconds();
	printf("Kernel MQDB product with dynamic parall. GRID(k)...\n");
	mqdbProdDPk<<< 1, k >>>(d_A, d_B, d_C, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime4 = seconds() - start;
	printf("   elapsed time:                        %.2f (sec)\n", GPUtime4);
	printf("   speedup vs GPU MQDB product:         %.2f\n", GPUtime2/GPUtime4);
	printf("   speedup vs GPU MQDB product GRID(1): %.2f\n", GPUtime3/GPUtime4);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	CHECK(cudaFree(d_A.elem));
	CHECK(cudaFree(d_B.elem));
	CHECK(cudaFree(d_C.elem));

	// collect times
	times->GPUtmsMQDB = GPUtime2;
	times->GPUtmsMQDBDynPar1 = GPUtime3;
	times->GPUtmsMQDBDynPark = GPUtime4;
	float den = 0;
	for (uint j = 0; j < k; j++)
		den += A.blkSize[j]*A.blkSize[j];
	times->density = den/(n*n);
}

/*
 * main function
 */
int main(int argc, char *argv[]) {
	uint n = 16*1024;      // matrix size
	uint min_k = 10;       // max num of blocks
	uint max_k = 20;       // max num of blocks

	struct tms times[max_k-min_k+1];

	// multiple tests on kernels
	for (uint k = min_k; k <= max_k; k++) {
		printf("\n*****   k = %d --- (avg block size = %f)\n",k,(float)n/k);
		testKernelsMQDB(n, k, &times[k-min_k]);
	}

	FILE *fd;
	fd = fopen("res.csv", "w");
	if (fd == NULL) {
		perror("file error!\n");
		exit(1);
	}

	// write results on file
	fprintf(fd,"num blocks,");
		for (uint j = 0; j <= max_k-min_k; j++)
			fprintf(fd,"%d,",j+min_k);

	fprintf(fd,"\nKernel MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDB);

	fprintf(fd,"\nKernel MQDB product with dynamic parall. GRID(1),");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDBDynPar1);

	fprintf(fd,"\nKernel MQDB product with dynamic parall. GRID(k),");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDBDynPark);

	fprintf(fd,"\ndensity,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].density);

	fclose(fd);

	return 0;
}




In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_70 -dc src/MQDB-CUDA-DP.cu GPUcomputing/lab1/MQDB/mqdb.cpp
!nvcc -arch=sm_70 MQDB-CUDA-DP.o mqdb.o -o mqdb_prod
!rm -rf *.o
!./mqdb_prod

### ↘️ *`TODO...`*

In [ ]:
%%cuda --name mqdb_DP.cu

#include "../../utils/MQDB/mqdb.h"

#define BLOCK_SIZE 16     // block size


/*
 * Kernel for block sub-matrix product of mqdb
 */
__global__ void mqdbBlockProd(mqdb A, mqdb B, mqdb C, uint sdim, uint d, uint n) {
	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	// jump to the right block sub-matrix
	int  offset = (n+1)*sdim;

	// each thread computes an entry of the product matrix
	if ((row < d) && (col < d)) {
		float val = 0;

		for (int k = 0; k < d; k++)
			val += A.elem[row * n + k + offset] * B.elem[k * n + col + offset];
		C.elem[row * n + col + offset] = val;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb: parent grid(1)
 */
__global__ void mqdbProdDP1(mqdb A, mqdb B, mqdb C, uint k, uint n) {
	// using grid(1,1)
	uint sdim = 0;
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	for (uint i = 0; i < k; i++ ) {
		uint d = A.blkSize[i];
		mqdbBlockProd<<<grid, block>>>(A, B, C, sdim, d, n);
		sdim += d;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb: parent grid(k)
 */
__global__ void mqdbProdDPk(mqdb A, mqdb B, mqdb C, uint n) {
	// using grid(1,k)
	uint i = threadIdx.x;
	uint sdim = 0;

	// block displacement
	uint d = A.blkSize[i];
	if (i > 0) {
		for (uint j = 0; j < i; j++)
			sdim += A.blkSize[j];
	}

	// grid dims
	dim3 grid((d + blockDim.x - 1) / blockDim.x, (d + blockDim.y - 1) / blockDim.y);
	mqdbBlockProd<<<grid,blockDim>>>(A, B, C, sdim, d, n);
}


In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37 -dc MQDB-CUDA-DP/mqdb_DP.cu MQDB-CUDA-DP/main.cu ../utils/MQDB/mqdb.cpp
!nvcc -arch=sm_37 mqdb_DP.o main.o mqdb.o -o mqdb_prod
!rm -rf *.o
!./mqdb_prod